In [ ]:
import urllib.parse
import urllib.request
import pandas as pd
import wget
import os
from tqdm import tqdm

# https://github.com/ronakvijay/Protein_Sequence_Classification

In [ ]:
url = 'https://www.uniprot.org/uploadlists/'
targets = pd.read_csv('data/DTI.csv')
fset = pd.DataFrame()

In [ ]:
# Map CHEMBL traget IDs to uniprot ID
for i in tqdm(range(len(targets['target'].unique()))):
    params = {
    'from': 'CHEMBL_ID',
    'to': 'ACC',
    'format': 'tab',
    'query': targets['target'].unique()[i]
    }

    data = urllib.parse.urlencode(params)
    data = data.encode('utf-8')
    req = urllib.request.Request(url, data)
    with urllib.request.urlopen(req) as f:
        response = f.read()
    a = response.decode('utf-8')

    with open("chembl2uniprot2.txt", "a") as file: 
        file.write('\n')
        file.write(a.split('\n')[1])
        #file.close()

In [ ]:
# Extract protin descriptors 
# ['AAC', 'CKSAAP', 'CTDC', 'CTDD', 'CTDT', 'DDE', 'DPC',
# 'GAAC', 'GDPC', 'GTPC', 'TPC', 'QSOrder', 'KSCTriad', 'CTriad']

chembl2uniprot = pd.read_csv('chembl2uniprot.txt', sep='\t', header=None)
des_type = 'AAC'
for CHEMBL_ID, uniprot_ID in zip(chembl2uniprot[0], chembl2uniprot[1]):
    print(CHEMBL_ID, uniprot_ID)
    push ='python F:/Projects/202004_DrugTargetNN/iFeature/iFeature.py --file %s --type %s'%('fasta/'+uniprot_ID+'.fasta', des_type)
    os.system(push)
    enc = pd.read_csv('encoding.tsv', delimiter='\t', encoding='utf-8')
    enc['uniprot_ID'] = uniprot_ID
    enc['CHEMBL_ID'] = CHEMBL_ID
    fset = fset.append(enc)
fset.to_csv('data/target_'+ des_type + '.csv')